This script joins exits with different filters and uploads data outputs of previous scripts to AGOL.

Exits subsets:

* Exits, tagged with flood hazard zone data
* Exits, tagged with distance from electric substation
* Exits, tagged with Euclidean distance to nearest DC fast charger location
* Exits, tagged with # of amenities within 1 mile
* Exits, tagged with Drive time distance to nearest DC fast charger location

*Yiwen Wang | 04/27/2020*


#### Import packages and create an authenticated link to your AGOL account

In [2]:
#Ignore warnings
import warnings
warnings.simplefilter('ignore')

In [3]:
#Load packages
from arcgis import GIS
import pandas as pd
from arcgis.features import GeoAccessor

In [4]:
#Authenticate our GIS object
gis = GIS(url='https://NSOE.maps.arcgis.com',username='yw377_790')

Enter password: ········


#### Import, tidy, and join datasets
Join each separate exit attribute to a single "master" feature class with FID.

In [5]:
#Read local datasets into spatial dataframes
sdf_Exits = GeoAccessor.from_featureclass('../Data/MJBA/Exits.shp')
sdf_floods = GeoAccessor.from_featureclass('../Data/MJBA/exits_flood.shp')
sdf_EucDist = GeoAccessor.from_featureclass('../Data/MJBA/Exits_distance_to_DCFC.shp')
sdf_AmenityCount = GeoAccessor.from_featureclass('../Data/MJBA/exits_amenities.shp')
sdf_DriveDist = GeoAccessor.from_featureclass('../Data/exits_distance.shp')
sdf_substations = GeoAccessor.from_featureclass('../Data/MJBA/Exits_distance_to_substations.shp')

In [6]:
#Subset fields from the Exits feature class and set FID to the index
sdf_Exits = sdf_Exits[['FID','Longitude', 'Latitude', 'County', 'State','SHAPE']].set_index('FID')

In [7]:
#Extract just the FID and meaningful attribute, setting the index to FID
sdf_floods = sdf_floods[['FID','Flood_Zone']].set_index('FID')
sdf_EucDist = sdf_EucDist[['FID','Dist_to_DC']].set_index('FID')
sdf_AmenityCount = sdf_AmenityCount[['FID','amenities#']].set_index('FID')
sdf_DriveDist = sdf_DriveDist[['FID','distance']].set_index('FID')
sdf_substations = sdf_substations[['FID','Dist_to_su']].set_index('FID')

In [8]:
#Merge all datasets into a single exit dataset
sdf_Master = pd.merge(left=sdf_Exits,right=sdf_floods,how='left',left_index=True,right_index=True)
sdf_Master = pd.merge(left=sdf_Master,right=sdf_EucDist,how='left',left_index=True,right_index=True)
sdf_Master = pd.merge(left=sdf_Master,right=sdf_AmenityCount,how='left',left_index=True,right_index=True)
sdf_Master = pd.merge(left=sdf_Master,right=sdf_DriveDist,how='left',left_index=True,right_index=True)
sdf_Master = pd.merge(left=sdf_Master,right=sdf_substations,how='left',left_index=True,right_index=True)

In [9]:
sdf_Master.head()

,Longitude,Latitude,County,State,SHAPE,Flood_Zone,Dist_to_DC,amenities#,distance,Dist_to_su
FID,,,,,,,,,,
7059,-84.314304,35.019492,Cherokee,NC,"{'x': -84.31430368976345, 'y': 35.019491849669...",Area of Minimal Flood Hazard,102,0,NaN,5
7060,-84.311492,35.014933,Cherokee,NC,"{'x': -84.31149165991212, 'y': 35.014933379571...",Area of Minimal Flood Hazard,102,0,NaN,5
7061,-84.295528,35.013561,Cherokee,NC,"{'x': -84.29552778977563, 'y': 35.013560589649...",Area of Minimal Flood Hazard,102,0,NaN,5
7062,-84.280291,35.012880,Cherokee,NC,"{'x': -84.28029090010494, 'y': 35.012879950349...",Area of Minimal Flood Hazard,101,0,NaN,4
7063,-84.275190,35.017816,Cherokee,NC,"{'x': -84.27519011992803, 'y': 35.017816120417...",Area of Minimal Flood Hazard,100,0,NaN,3


In [10]:
#Rename columns
sdf_Master.rename(columns={'Dist_to_DC':'Mi_to_DCFC',
                           'distance':'RdMi_to_DCFC',
                           'Dist_to_su':'Dist_to_substations'},inplace=True)

In [58]:
#Optional: Save to a new local shapefile
sdf_Master.spatial.to_featureclass('../Data/Exits_final.shp')

'../Data/Exits_final.shp'

### Publish datasets to ArcGIS Online account

* the NREL DCFC charging sites
* the extracted OSM amenities locations
* attributed candidate sites (i.e. the exits)
* NC road network

#### upload NREL DCFC charging sites

In [19]:
#Fetch the NREL CSV into a pandas dataframe, then convert to a spatial dataframe
DCFC_df = pd.read_csv('../Data/NREL/DCFC.csv')
DCFC_sdf = GeoAccessor.from_xy(df = DCFC_df, x_column='longitude', y_column='latitude',sr=4326)

In [20]:
#Convert the sdf to a FeatureLayer, which publishes it to AGOL
DCFC_flyr = DCFC_sdf.spatial.to_featurelayer(title='NREL DC Fast Charger sites',
                                             tags='EV Charger,ENV790')
DCFC_flyr

<Item title:"NREL DC Fast Charger sites" type:Feature Layer Collection owner:yw377_790>

#### Update properties for the layer

1. creating a dictionary of properties and their values 
2. updating our feature layer with these properties. 
[*https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.Item.update*]

In [21]:
#Construct the dictionary of feature layer properties
featLayer_properties = {
    'snippet':"NREL DC Fast Charging site. Extracted and subset via the NREL API",
    'summary':"These data were extracted from the NREL API, subset using the following criteria..."
}
#Apply the changes
DCFC_flyr.update(featLayer_properties)

True

In [23]:
#Share with everyone in the organization
DCFC_flyr.share(org=True)

{'results': [{'itemId': 'bb59d50a32834fef810f5bca2a187380',
   'success': True,
   'notSharedWith': []}]}

#### Upload OSM amenities datasets

In [61]:
sdf_amenities = GeoAccessor.from_featureclass('../Data/OSM/OSM_amenities.shp')
amenities_flyr = sdf_amenities.spatial.to_featurelayer(title='Restaurants and cafes locaitons in NC sites',
                                             tags='Amenities,ENV790')
amenities_flyr
amenities_flyr.share(org=True)

<Item title:"Restaurants and cafes locaitons in NC sites" type:Feature Layer Collection owner:yw377_790>

#### Upload attributed candidate charger locations

In [63]:
sdf_charger = GeoAccessor.from_featureclass('../Data/Exits_final.shp')
charger_flyr = sdf_charger.spatial.to_featurelayer(title='Attributed candidate charger locations',
                                             tags='EV Charger,ENV790')
charger_flyr
charger_flyr.share(org=True)

{'results': [{'itemId': '33aaba833eb643f2a761bed9654515f3',
   'success': True,
   'notSharedWith': []}]}

#### Upload NC road network

In [11]:
#road network edges
sdf_edges = GeoAccessor.from_featureclass('../Data/OSM/NC_roads/edges/edges.shp')
edges_flyr = sdf_edges.spatial.to_featurelayer(title='NC road network - edges',
                                             tags='EV Charger,ENV790')
edges_flyr
edges_flyr.share(org=True)

#road network nodes
sdf_nodes = GeoAccessor.from_featureclass('../Data/OSM/NC_roads/nodes/nodes.shp')
nodes_flyr = sdf_nodes.spatial.to_featurelayer(title='NC road network - nodes',
                                             tags='EV Charger,ENV790')
nodes_flyr
nodes_flyr.share(org=True)

{'results': [{'itemId': 'fdb83ce47e644cfebe13ee57a8766509',
   'success': True,
   'notSharedWith': []}]}